In [1]:
import os

In [2]:
%pwd   # as it shows route directory.

'd:\\NLP_Project\\Text-Summarizer-project\\research'

In [3]:
os.chdir("../")  # its change directory command. and i want to move to
                 #Text-Summarizer-project folder from research folder.

In [4]:
%pwd

'd:\\NLP_Project\\Text-Summarizer-project'

In [5]:
# 3. Update entity
# lets create an entity . its return type of a function.
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml

In [7]:
from textSummarizer.utils.common import create_directories

In [9]:
# 4. Update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # i am taking it from textSummarizer.constants.
        params_filepath = PARAMS_FILE_PATH): # i am taking it from textSummarizer.constants too.

        self.config = read_yaml(config_filepath) # now i am passing this path in read_yaml.and it will read all the
        self.params = read_yaml(params_filepath) # data from the config.yaml file.and i can access all the variables
        create_directories([self.config.artifacts_root]) # by calling create_directories function we will 
     # pass artifacts_root from config.yaml file. It acts like configBox like output. as in eg. trials.ipynb


    

    def get_data_ingestion_config(self) -> DataIngestionConfig: # its a return type of above previous cell.
        config = self.config.data_ingestion    # these are variables below it will return.
    
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    # so we have created our custom return type of any function using this data class and we call it entity.
    # And this is my config manager(class) inside that i have my data ingestion related to configuration.

    # THE MOST IMPORTANT THING WE DONT NEED TO CHANGE ANY THING HERE. IF WE NEED WE JUST MAKE CHANGE IN config.yaml
    # AND IT REFLECT HERE AUTOMATICALLLY.

In [11]:
# 5. update the components
import os
import urllib.request as request # Using it to download my data from url
import zipfile                   # to perform unzip operation
from textSummarizer.logging import logger  # import from common.py file
from textSummarizer.utils.common import get_size  # import from common.py file

In [12]:
# I have creatd one class named DataIngestionConfig
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config   # here i am initializing the config

# two methods 1. download data from url and 2. unzip the file
    
    def download_file(self):         
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,  # its the url we defined in config.yml
                filename = self.config.local_data_file  # it shows where i am downloading this file(local_data_file).
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
          # i also want to see the size of the file. therefore using get_size method.
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir # unzip into unzip_dir.
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  # i am extracting in unzip file

In [13]:
#  first i will handle try except block
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-13 15:30:02,680: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2023-11-13 15:30:02,711: INFO: common: yaml file: params.yaml loaded successfully:]
[2023-11-13 15:30:02,717: INFO: common: created directory at: artifacts:]
[2023-11-13 15:30:02,724: INFO: common: created directory at: artifacts/data_ingestion:]
[2023-11-13 15:30:27,853: INFO: 2572867094: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7050697
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e91f78c406218c4bb71a6b5ac4bcc7a806eeec112f4ed4d24a8f9366e8d40c2f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 528A:5E4D:275D4D4:2952FE1:6551FAB4
Accept-Ranges: bytes
Date: Mon, 13 Nov 2023 10:30:13 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230126-FRA